In [1]:
!pip install -q kaggle
!pip install -q nibabel
!pip install -q scikit-learn
!pip install -q tqdm
!pip install -q split-folders
!pip install -q torchinfo
!pip install -q segmentation-models-pytorch-3d
!pip install -q livelossplot
!pip install -q torchmetrics
!pip install -q tensorboard
!pip install -q matplotlib seaborn plotly
!pip install -q opencv-python-headless
!pip install -q pandas numpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.0/121.0 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 52.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 626.8/626.8 kB 39.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.2/983.2 kB 20.6 MB/s eta 0:00:00


In [2]:
import os
import random
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import nibabel as nib
import cv2
from tqdm import tqdm
import splitfolders
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW, Adam
from torch.optim.lr_scheduler import CosineAnnealingLR, ReduceLROnPlateau
import torchvision.transforms as transforms
from torch.cuda import amp
from torchmetrics import MeanMetric, Dice, JaccardIndex
from torchmetrics.classification import MulticlassAccuracy, MulticlassPrecision, MulticlassRecall
import torchinfo
import segmentation_models_pytorch_3d as smp
from livelossplot import PlotLosses
from livelossplot.outputs import MatplotlibPlot
import warnings
warnings.filterwarnings('ignore')

ImportError: cannot import name 'Dice' from 'torchmetrics' (/usr/local/lib/python3.12/dist-packages/torchmetrics/__init__.py)

In [ ]:
def seed_everything(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seed_everything(42)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")


In [ ]:
from google.colab import files

print("Please upload your Kaggle API credentials (kaggle.json)")
print("Instructions: Go to https://www.kaggle.com/account → Create API Token")
files.upload()

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [4]:
print("Downloading BraTS 2021 dataset...")
!kaggle datasets download -d dschettler8845/brats-2021-task1
!unzip -q brats-2021-task1.zip -d /content/BraTS2021/
!rm -f brats-2021-task1.zip
DATASET_PATH = "/content/BraTS2021/"
print(f"Dataset extracted to: {DATASET_PATH}")
print(f"Total files in dataset: {len(os.listdir(DATASET_PATH))}"

SyntaxError: incomplete input (ipython-input-735401617.py, line 7)

In [6]:
t1_list = sorted(glob.glob(f"{DATASET_PATH}/BraTS2021_*/*/*t1.nii"))
t1ce_list = sorted(glob.glob(f"{DATASET_PATH}/BraTS2021_*/*/*t1ce.nii"))
t2_list = sorted(glob.glob(f"{DATASET_PATH}/BraTS2021_*/*/*t2.nii"))
flair_list = sorted(glob.glob(f"{DATASET_PATH}/BraTS2021_*/*/*flair.nii"))
mask_list = sorted(glob.glob(f"{DATASET_PATH}/BraTS2021_*/*/*seg.nii"))
print(f"Found files:")
print(f"T1: {len(t1_list)}")
print(f"T1ce: {len(t1ce_list)}")
print(f"T2: {len(t2_list)}")
print(f"FLAIR: {len(flair_list)}")
print(f"Masks: {len(mask_list)}")

NameError: name 'DATASET_PATH' is not defined

In [2]:
def preprocess_data():
    """
    Preprocesses BraTS 2021 data: scales, crops, and saves as numpy arrays.
    Implements ROI cropping to reduce computational requirements.
    """
    scaler = MinMaxScaler()
    os.makedirs("BraTS2021_Preprocessed/images", exist_ok=True)
    os.makedirs("BraTS2021_Preprocessed/masks", exist_ok=True)
  for idx in tqdm(range(min(len(t1_list), 500)), desc="Preprocessing samples"):
        try:

            t1 = nib.load(t1_list[idx]).get_fdata()
            t1ce = nib.load(t1ce_list[idx]).get_fdata()
            t2 = nib.load(t2_list[idx]).get_fdata()
            flair = nib.load(flair_list[idx]).get_fdata()
            mask = nib.load(mask_list[idx]).get_fdata().astype(np.uint8)


            t1 = scaler.fit_transform(t1.reshape(-1, 1)).reshape(t1.shape)
            t1ce = scaler.fit_transform(t1ce.reshape(-1, 1)).reshape(t1ce.shape)
            t2 = scaler.fit_transform(t2.reshape(-1, 1)).reshape(t2.shape)
            flair = scaler.fit_transform(flair.reshape(-1, 1)).reshape(flair.shape)


            combined_images = np.stack([flair, t1ce, t2, t1], axis=3)


            combined_images = combined_images[56:184, 56:184, 13:141]
            mask = mask[56:184, 56:184, 13:141]


            tumor_pixels = np.sum(mask > 0)
            total_pixels = mask.size
            tumor_ratio = tumor_pixels / total_pixels

            if tumor_ratio > 0.01:
                np.save(f"BraTS2021_Preprocessed/images/image_{idx:04d}.npy", combined_images)
                np.save(f"BraTS2021_Preprocessed/masks/mask_{idx:04d}.npy", mask)
                valid_samples += 1
              except Exception as e:
            print(f"Error processing sample {idx}: {e}")
            continue

    print(f"Preprocessing complete. Valid samples: {valid_samples}")
    return valid_samples

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 9)

In [ ]:
num_valid_samples = preprocess_data()

In [ ]:
input_folder = "BraTS2021_Preprocessed/"
output_folder = "BraTS2021_Split/"

splitfolders.ratio(
    input_folder,
    output=output_folder,
    seed=42,
    ratio=(0.8, 0.2),
    group_prefix=None
)

print("Data split completed:")
print(f"Train: {len(os.listdir(os.path.join(output_folder, 'train/images')))} samples")
print(f"Val: {len(os.listdir(os.path.join(output_folder, 'val/images')))} samples")

In [ ]:
class BraTSDataset(Dataset):
    """
    Custom Dataset for BraTS 2021 brain tumor segmentation.
    Handles 3D medical image loading and preprocessing.
    """

    def __init__(self, img_dir, mask_dir, normalize=True, augment=False):
        self.img_dir = img_dir
        self.mask_dir = mask_dir
        self.normalize = normalize
        self.augment = augment

        self.img_list = sorted(os.listdir(img_dir))
        self.mask_list = sorted(os.listdir(mask_dir))


        if self.normalize:
            self.mean = torch.tensor([0.5, 0.5, 0.5, 0.5]).view(4, 1, 1, 1)
            self.std = torch.tensor([0.5, 0.5, 0.5, 0.5]).view(4, 1, 1, 1)

    def __len__(self):
        return len(self.img_list)

    def __getitem__(self, idx):

        img_path = os.path.join(self.img_dir, self.img_list[idx])
        mask_path = os.path.join(self.mask_dir, self.mask_list[idx])

        image = np.load(img_path)
        mask = np.load(mask_path)

        image = torch.from_numpy(image).permute(3, 2, 0, 1).float()
        mask = torch.from_numpy(mask).long()


        if self.normalize:
            image = (image - self.mean) / self.std


        if self.augment:
            image, mask = self.augment_data(image, mask)

        return image, mask

    def augment_data(self, image, mask):
        """Simple augmentation strategies for medical images"""
        if torch.rand(1) > 0.5:
            angle = torch.rand(1) * 0.1 - 0.05
            image = self.rotate_3d(image, angle)
            mask = self.rotate_3d(mask.float(), angle).long()

        if torch.rand(1) > 0.5:
            noise = torch.randn_like(image) * 0.01
            image = image + noise

        return image, mask

    def rotate_3d(self, tensor, angle):
        """Simple 3D rotation (placeholder - in practice, use specialized libraries)"""
        return tensor


In [ ]:
train_img_dir = os.path.join(output_folder, "train/images")
train_mask_dir = os.path.join(output_folder, "train/masks")
val_img_dir = os.path.join(output_folder, "val/images")
val_mask_dir = os.path.join(output_folder, "val/masks")

train_dataset = BraTSDataset(train_img_dir, train_mask_dir, normalize=True, augment=True)
val_dataset = BraTSDataset(val_img_dir, val_mask_dir, normalize=True, augment=False)


In [ ]:
BATCH_SIZE = 2  # Reduced for Colab memory constraints
train_loader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=2,
    pin_memory=True
)
val_loader = DataLoader(
    val_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=2,
    pin_memory=True
)

In [ ]:
sample_image, sample_mask = train_dataset[0]
print(f"Sample image shape: {sample_image.shape}")
print(f"Sample mask shape: {sample_mask.shape}")
print(f"Unique mask values: {torch.unique(sample_mask)}")


In [ ]:

class DoubleConv(nn.Module):
    """Double convolution block with batch normalization and dropout"""

    def __init__(self, in_channels, out_channels, dropout_rate=0.1):
        super(DoubleConv, self).__init__()
        self.double_conv = nn.Sequential(
            nn.Conv3d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm3d(out_channels),
            nn.ReLU(inplace=True),
            nn.Dropout3d(dropout_rate),
            nn.Conv3d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm3d(out_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.double_conv(x)

class UNet3D(nn.Module):
    """Standard 3D U-Net for brain tumor segmentation"""

    def __init__(self, in_channels=4, out_channels=4, features=[16, 32, 64, 128, 256]):
        super(UNet3D, self).__init__()
        self.ups = nn.ModuleList()
        self.downs = nn.ModuleList()
        self.pool = nn.MaxPool3d(kernel_size=2, stride=2)

        # Encoder (downsampling path)
        in_ch = in_channels
        for feature in features:
            self.downs.append(DoubleConv(in_ch, feature))
            in_ch = feature

        # Bottleneck
        self.bottleneck = DoubleConv(features[-1], features[-1] * 2)

        # Decoder (upsampling path)
        for feature in reversed(features):
            self.ups.append(
                nn.ConvTranspose3d(
                    feature * 2, feature, kernel_size=2, stride=2
                )
            )
            self.ups.append(DoubleConv(feature * 2, feature))

        # Final classification layer
        self.final_conv = nn.Conv3d(features[0], out_channels, kernel_size=1)

    def forward(self, x):
        skip_connections = []

        # Encoder
        for down in self.downs:
            x = down(x)
            skip_connections.append(x)
            x = self.pool(x)


        x = self.bottleneck(x)
        skip_connections = skip_connections[::-1]

        # Decoder
        for idx in range(0, len(self.ups), 2):
            x = self.ups[idx](x)
            skip_connection = skip_connections[idx // 2]

            if x.shape != skip_connection.shape:
                x = F.interpolate(x, size=skip_connection.shape[2:])

            concat_skip = torch.cat((skip_connection, x), dim=1)
            x = self.ups[idx + 1](concat_skip)

        return self.final_conv(x)

In [ ]:

model = UNet3D(in_channels=4, out_channels=4)
model = model.to(device)

In [ ]:
print("3D U-Net Architecture Summary:")
print("=" * 50)
print(torchinfo.summary(model, input_size=(1, 4, 128, 128, 128)))

In [ ]:
with torch.no_grad():
    test_input = torch.randn(1, 4, 128, 128, 128).to(device)
    test_output = model(test_input)
    print(f"\nTest Input Shape: {test_input.shape}")
    print(f"Test Output Shape: {test_output.shape}")


In [ ]:
class DiceLoss(nn.Module):
    """Dice Loss for medical image segmentation"""

    def __init__(self, smooth=1e-6):
        super(DiceLoss, self).__init__()
        self.smooth = smooth

    def forward(self, pred, target):
        pred = torch.softmax(pred, dim=1)
        target = F.one_hot(target, num_classes=4).permute(0, 4, 1, 2, 3).float()

        intersection = (pred * target).sum()
        union = pred.sum() + target.sum()
        dice = (2. * intersection + self.smooth) / (union + self.smooth)

        return 1 - dice.mean()

class FocalLoss(nn.Module):
    """Focal Loss for addressing class imbalance"""

    def __init__(self, alpha=1, gamma=2, smooth=1e-6):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.smooth = smooth

    def forward(self, pred, target):
        ce_loss = F.cross_entropy(pred, target, reduction='none')
        pt = torch.exp(-ce_loss)
        focal_loss = self.alpha * (1 - pt) ** self.gamma * ce_loss

        return focal_loss.mean()

class TverskyLoss(nn.Module):
    """Tversky Loss for addressing false positives and false negatives"""

    def __init__(self, alpha=0.7, beta=0.3, smooth=1e-6):
        super(TverskyLoss, self).__init__()
        self.alpha = alpha
        self.beta = beta
        self.smooth = smooth

    def forward(self, pred, target):
        pred = torch.softmax(pred, dim=1)
        target = F.one_hot(target, num_classes=4).permute(0, 4, 1, 2, 3).float()

        tp = (pred * target).sum()
        fp = pred * (1 - target)
        fn = (1 - pred) * target

        tversky = (tp + self.smooth) / (tp + self.alpha * fp + self.beta * fn + self.smooth)

        return 1 - tversky.mean()

class FocalTverskyLoss(nn.Module):
    """Focal Tversky Loss combining focal and Tversky losses"""

    def __init__(self, alpha=0.7, beta=0.3, gamma=4/3, smooth=1e-6):
        super(FocalTverskyLoss, self).__init__()
        self.alpha = alpha
        self.beta = beta
        self.gamma = gamma
        self.smooth = smooth

    def forward(self, pred, target):
        pred = torch.softmax(pred, dim=1)
        target = F.one_hot(target, num_classes=4).permute(0, 4, 1, 2, 3).float()

        tp = (pred * target).sum()
        fp = pred * (1 - target)
        fn = (1 - pred) * target

        tversky = (tp + self.smooth) / (tp + self.alpha * fp + self.beta * fn + self.smooth)
        focal_tversky = torch.pow(1 - tversky, self.gamma)

        return focal_tversky.mean()

class UnifiedFocalLoss(nn.Module):
    """Unified Focal Loss combining Dice and Focal losses"""

    def __init__(self, dice_weight=0.5, focal_weight=0.5, smooth=1e-6):
        super(UnifiedFocalLoss, self).__init__()
        self.dice_weight = dice_weight
        self.focal_weight = focal_weight
        self.smooth = smooth
        self.dice_loss = DiceLoss(smooth)
        self.focal_loss = FocalLoss()

    def forward(self, pred, target):
        dice = self.dice_loss(pred, target)
        focal = self.focal_loss(pred, target)

        return self.dice_weight * dice + self.focal_weight * focal

class CombinedTverskyLoss(nn.Module):
    """Combined Tversky and Focal Loss"""

    def __init__(self, tversky_weight=0.7, focal_weight=0.3, alpha=0.7, beta=0.3, gamma=4/3):
        super(CombinedTverskyLoss, self).__init__()
        self.tversky_weight = tversky_weight
        self.focal_weight = focal_weight
        self.tversky_loss = TverskyLoss(alpha=alpha, beta=beta)
        self.focal_loss = FocalLoss(alpha=0.25, gamma=2.0)

    def forward(self, pred, target):
        tversky = self.tversky_loss(pred, target)
        focal = self.focal_loss(pred, target)

        return self.tversky_weight * tversky + self.focal_weight * focal

In [ ]:
LOSS_FUNCTIONS = {
    'dice': DiceLoss(),
    'focal': FocalLoss(),
    'tversky': TverskyLoss(),
    'focal_tversky': FocalTverskyLoss(),
    'unified_focal': UnifiedFocalLoss(),
    'combined_tversky': CombinedTverskyLoss(),
    'dice_focal': CombinedTverskyLoss(tversky_weight=0.6, focal_weight=0.4),
    'ce_dice': None  # Will implement below
}


In [ ]:
class CEDiceLoss(nn.Module):
    def __init__(self, ce_weight=0.5, dice_weight=0.5):
        super(CEDiceLoss, self).__init__()
        self.ce_weight = ce_weight
        self.dice_weight = dice_weight
        self.ce_loss = nn.CrossEntropyLoss()
        self.dice_loss = DiceLoss()

    def forward(self, pred, target):
        ce = self.ce_loss(pred, target)
        dice = self.dice_loss(pred, target)

        return self.ce_weight * ce + self.dice_weight * dice

LOSS_FUNCTIONS['ce_dice'] = CEDiceLoss()

print("Implemented Loss Functions:")
print("=" * 30)
for loss_name in LOSS_FUNCTIONS.keys():
    print(f"- {loss_name}")

In [ ]:

class TrainingConfig:
    """Training configuration"""
    BATCH_SIZE = 2
    EPOCHS = 50
    LEARNING_RATE = 1e-3
    WEIGHT_DECAY = 1e-4
    SAVE_DIR = "model_checkpoints"
    NUM_CLASSES = 4
    INPUT_CHANNELS = 4

# Create save directory
os.makedirs(TrainingConfig.SAVE_DIR, exist_ok=True)

class MetricsCalculator:
    """Calculate comprehensive metrics for segmentation evaluation"""

    def __init__(self, num_classes=4):
        self.num_classes = num_classes
        self.dice_metric = Dice(num_classes=num_classes, average='macro')
        self.iou_metric = JaccardIndex(num_classes=num_classes, average='macro')
        self.accuracy_metric = MulticlassAccuracy(num_classes=num_classes, average='macro')
        self.precision_metric = MulticlassPrecision(num_classes=num_classes, average='macro')
        self.recall_metric = MulticlassRecall(num_classes=num_classes, average='macro')

        self.reset()

    def reset(self):
        """Reset all metrics"""
        self.dice_metric.reset()
        self.iou_metric.reset()
        self.accuracy_metric.reset()
        self.precision_metric.reset()
        self.recall_metric.reset()

    def update(self, pred, target):
        """Update metrics with predictions and targets"""
        pred.argmax(dim=1)
        self.dice_metric.update(pred, target)
        self.iou_metric.update(pred, target)
        self.accuracy_metric.update(pred, target)
        self.precision_metric.update(pred, target)
        self.recall_metric.update(pred, target)

    def compute(self):
        """Compute all metrics"""
        return {
            'dice': self.dice_metric.compute().item(),
            'iou': self.iou_metric.compute().item(),
            'accuracy': self.accuracy_metric.compute().item(),
            'precision': self.precision_metric.compute().item(),
            'recall': self.recall_metric.compute().item()
        }

def train_epoch(model, loader, loss_fn, optimizer, scheduler, scaler, metrics_calc):
    """Train for one epoch"""
    model.train()
    metrics_calc.reset()
    total_loss = 0

    for batch_idx, (data, target) in enumerate(loader):
        data, target = data.to(device), target.to(device)

        optimizer.zero_grad()

        # Mixed precision training for memory efficiency
        with amp.autocast():
            output = model(data)
            loss = loss_fn(output, target)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        if scheduler is not None:
            scheduler.step()

        total_loss += loss.item()
        metrics_calc.update(output.detach(), target)

        if batch_idx % 10 == 0:
            print(f'Batch {batch_idx}/{len(loader)}, Loss: {loss.item():.4f}')

    avg_loss = total_loss / len(loader)
    metrics = metrics_calc.compute()

    return avg_loss, metrics

def validate_epoch(model, loader, loss_fn, metrics_calc):
    """Validate for one epoch"""
    model.eval()
    metrics_calc.reset()
    total_loss = 0

    with torch.no_grad():
        for data, target in loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            loss = loss_fn(output, target)

            total_loss += loss.item()
            metrics_calc.update(output, target)

    avg_loss = total_loss / len(loader)
    metrics = metrics_calc.compute()

    return avg_loss, metrics

In [ ]:
def train_with_loss_function(loss_name, loss_fn, epochs=50):
    """Train model with specific loss function"""
    print(f"\n{'='*60}")
    print(f"Training with {loss_name} loss function")
    print(f"{'='*60}")


    model = UNet3D(in_channels=4, out_channels=4).to(device)
    optimizer = AdamW(model.parameters(), lr=TrainingConfig.LEARNING_RATE, weight_decay=TrainingConfig.WEIGHT_DECAY)
    scheduler = CosineAnnealingLR(optimizer, T_max=epochs)
    scaler = amp.GradScaler()
    metrics_calc = MetricsCalculator()


    train_losses = []
    val_losses = []
    train_metrics = []
    val_metrics = []
    best_val_dice = 0
    best_model_state = None

    for epoch in range(epochs):
        print(f"\nEpoch {epoch+1}/{epochs}")
        print("-" * 30)


        train_loss, train_metrics_dict = train_epoch(model, train_loader, loss_fn, optimizer, scheduler, scaler, metrics_calc)
        train_losses.append(train_loss)
        train_metrics.append(train_metrics_dict)


        val_loss, val_metrics_dict = validate_epoch(model, val_loader, loss_fn, metrics_calc)
        val_losses.append(val_loss)
        val_metrics.append(val_metrics_dict)


        print(f"Train Loss: {train_loss:.4f}, Train Dice: {train_metrics_dict['dice']:.4f}")
        print(f"Val Loss: {val_loss:.4f}, Val Dice: {val_metrics_dict['dice']:.4f}")


        if val_metrics_dict['dice'] > best_val_dice:
            best_val_dice = val_metrics_dict['dice']
            best_model_state = model.state_dict().copy()


    model.load_state_dict(best_model_state)


    final_train_metrics = train_metrics[-1]
    final_val_metrics = val_metrics[-1]

    results = {
        'loss_name': loss_name,
        'best_val_dice': best_val_dice,
        'final_train_dice': final_train_metrics['dice'],
        'final_val_dice': final_val_metrics['dice'],
        'final_train_iou': final_train_metrics['iou'],
        'final_val_iou': final_val_metrics['iou'],
        'final_train_acc': final_train_metrics['accuracy'],
        'final_val_acc': final_val_metrics['accuracy'],
        'train_losses': train_losses,
        'val_losses': val_losses,
        'train_metrics_history': train_metrics,
        'val_metrics_history': val_metrics,
        'model_state': best_model_state
    }

    print(f"\n{loss_name} Results:")
    print(f"Best Val Dice: {best_val_dice:.4f}")
    print(f"Final Val Metrics - Dice: {final_val_metrics['dice']:.4f}, IoU: {final_val_metrics['iou']:.4f}")

    return results


print("Starting comprehensive loss function comparison...")
print("This will train the 3D U-Net with each loss function and compare results.")


all_results = {}
results_summary = []


EPOCHS_COMPARISON = 25

for loss_name, loss_fn in LOSS_FUNCTIONS.items():
    print(f"\n\nTraining with {loss_name}...")
    try:
        results = train_with_loss_function(loss_name, loss_fn, EPOCHS_COMPARISON)
        all_results[loss_name] = results

        results_summary.append({
            'Loss Function': loss_name,
            'Best Val Dice': results['best_val_dice'],
            'Final Val Dice': results['final_val_dice'],
            'Final Val IoU': results['final_val_iou'],
            'Final Val Accuracy': results['final_val_acc']
        })

    except Exception as e:
        print(f"Error training with {loss_name}: {e}")
        continue

In [ ]:
results_df = pd.DataFrame(results_summary)
results_df = results_df.sort_values('Best Val Dice', ascending=False)

print("\n" + "="*80)
print("LOSS FUNCTION COMPARISON RESULTS")
print("="*80)
print(results_df.round(4))

# Identify best loss function
best_loss_name = results_df.iloc[0]['Loss Function']
best_loss_result = all_results[best_loss_name]

print(f"\n🏆 BEST PERFORMING LOSS FUNCTION: {best_loss_name}")
print(f"Best Validation Dice Score: {best_loss_result['best_val_dice']:.4f}")
print(f"Final Validation Metrics:")
print(f"  - Dice: {best_loss_result['final_val_dice']:.4f}")
print(f"  - IoU: {best_loss_result['final_val_iou']:.4f}")
print(f"  - Accuracy: {best_loss_result['final_val_acc']:.4f}")

# Plot comparison metrics
plt.figure(figsize=(15, 12))

# Loss curves comparison
plt.subplot(2, 2, 1)
for loss_name, results in all_results.items():
    plt.plot(results['train_losses'], label=f'{loss_name} (train)', alpha=0.7)
    plt.plot(results['val_losses'], label=f'{loss_name} (val)', alpha=0.7, linestyle='--')
plt.title('Training and Validation Losses')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.yscale('log')

# Dice score comparison
plt.subplot(2, 2, 2)
dice_scores = [results['best_val_dice'] for results in all_results.values()]
loss_names = list(all_results.keys())
bars = plt.bar(loss_names, dice_scores)
plt.title('Best Validation Dice Scores')
plt.xlabel('Loss Function')
plt.ylabel('Dice Score')
plt.xticks(rotation=45)

# Color the best performing bar
best_idx = loss_names.index(best_loss_name)
bars[best_idx].set_color('gold')

# IoU comparison
plt.subplot(2, 2, 3)
iou_scores = [results['final_val_iou'] for results in all_results.values()]
bars = plt.bar(loss_names, iou_scores)
plt.title('Final Validation IoU Scores')
plt.xlabel('Loss Function')
plt.ylabel('IoU Score')
plt.xticks(rotation=45)
bars[best_idx].set_color('gold')

# Accuracy comparison
plt.subplot(2, 2, 4)
acc_scores = [results['final_val_acc'] for results in all_results.values()]
bars = plt.bar(loss_names, acc_scores)
plt.title('Final Validation Accuracy')
plt.xlabel('Loss Function')
plt.ylabel('Accuracy')
plt.xticks(rotation=45)
bars[best_idx].set_color('gold')

plt.tight_layout()
plt.savefig('loss_function_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

# Detailed analysis of best loss function
plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
plt.plot(best_loss_result['train_losses'], label='Train', color='blue')
plt.plot(best_loss_result['val_losses'], label='Validation', color='red')
plt.title(f'{best_loss_name} - Loss Curves')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.yscale('log')

plt.subplot(1, 3, 2)
train_dice = [m['dice'] for m in best_loss_result['train_metrics_history']]
val_dice = [m['dice'] for m in best_loss_result['val_metrics_history']]
plt.plot(train_dice, label='Train Dice', color='blue')
plt.plot(val_dice, label='Validation Dice', color='red')
plt.title(f'{best_loss_name} - Dice Score')
plt.xlabel('Epoch')
plt.ylabel('Dice Score')
plt.legend()

plt.subplot(1, 3, 3)
train_iou = [m['iou'] for m in best_loss_result['train_metrics_history']]
val_iou = [m['iou'] for m in best_loss_result['val_metrics_history']]
plt.plot(train_iou, label='Train IoU', color='blue')
plt.plot(val_iou, label='Validation IoU', color='red')
plt.title(f'{best_loss_name} - IoU Score')
plt.xlabel('Epoch')
plt.ylabel('IoU')
plt.legend()

plt.tight_layout()
plt.savefig('best_loss_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

def visualize_predictions(model, loader, num_samples=6):
    """Visualize model predictions on validation data"""
    model.eval()

    fig, axes = plt.subplots(num_samples, 5, figsize=(20, 4*num_samples))

    with torch.no_grad():
        for idx, (data, target) in enumerate(loader):
            if idx >= num_samples:
                break

            data, target = data.to(device), target.to(device)
            output = model(data)
            pred = torch.softmax(output, dim=1).argmax(dim=1)

            # Get middle slice for visualization
            middle_slice = data.shape[2] // 2

            # Original modalities (FLAIR as example)
            axes[idx, 0].imshow(data[idx, 0, middle_slice].cpu().numpy(), cmap='gray')
            axes[idx, 0].set_title('FLAIR')

            # Ground truth
            axes[idx, 1].imshow(target[idx, middle_slice].cpu().numpy(), cmap='tab10')
            axes[idx, 1].set_title('Ground Truth')

            # Prediction
            axes[idx, 2].imshow(pred[idx, middle_slice].cpu().numpy(), cmap='tab10')
            axes[idx, 2].set_title('Prediction')

            # Overlay
            overlay = data[idx, 0, middle_slice].cpu().numpy()
            axes[idx, 3].imshow(overlay, cmap='gray', alpha=0.7)
            axes[idx, 3].imshow(pred[idx, middle_slice].cpu().numpy(), cmap='tab10', alpha=0.3)
            axes[idx, 3].set_title('Overlay')

            # Metrics for this sample
            sample_metrics = MetricsCalculator()
            sample_metrics.update(output, target)
            metrics = sample_metrics.compute()

            axes[idx, 4].axis('off')
            axes[idx, 4].text(0.1, 0.8, f'Sample {idx+1}', fontsize=12, fontweight='bold')
            axes[idx, 4].text(0.1, 0.6, f'Dice: {metrics["dice"]:.3f}', fontsize=10)
            axes[idx, 4].text(0.1, 0.5, f'IoU: {metrics["iou"]:.3f}', fontsize=10)
            axes[idx, 4].text(0.1, 0.4, f'Acc: {metrics["accuracy"]:.3f}', fontsize=10)
            axes[idx, 4].text(0.1, 0.3, f'Prec: {metrics["precision"]:.3f}', fontsize=10)
            axes[idx, 4].text(0.1, 0.2, f'Recall: {metrics["recall"]:.3f}', fontsize=10)

    plt.tight_layout()
    plt.savefig('predictions_visualization.png', dpi=300, bbox_inches='tight')
    plt.show()

# Load best model and visualize predictions
best_model = UNet3D(in_channels=4, out_channels=4).to(device)
best_model.load_state_dict(best_loss_result['model_state'])

print(f"\nVisualizing predictions with {best_loss_name} loss function...")
visualize_predictions(best_model, val_loader, num_samples=4)

In [ ]:
best_model_path = f"{TrainingConfig.SAVE_DIR}/best_3d_unet_{best_loss_name}.pth"
torch.save({
    'model_state_dict': best_loss_result['model_state'],
    'loss_function': best_loss_name,
    'training_config': {
        'epochs': EPOCHS_COMPARISON,
        'learning_rate': TrainingConfig.LEARNING_RATE,
        'batch_size': TrainingConfig.BATCH_SIZE
    },
    'performance_metrics': {
        'best_val_dice': best_loss_result['best_val_dice'],
        'final_val_dice': best_loss_result['final_val_dice'],
        'final_val_iou': best_loss_result['final_val_iou'],
        'final_val_accuracy': best_loss_result['final_val_acc']
    }
}, best_model_path)

print(f"\nBest model saved to: {best_model_path}")